In [1]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn import metrics
import matplotlib.cm as cm
import pandas as pd
import numpy as np
import random
import nltk
import re

In [2]:
# import data
df = pd.read_csv('dara.csv')  

In [3]:
df = df.rename(columns={'0': 'title', '1': 'body', '2': 'tags'})

In [4]:
df

,Title,Body,Tags
0,efficient way to implement paging,<p>Should I use LINQ's <code>Skip()</code> and...,<sql><sql-server><asp.net-mvc><linq-to-sql><pa...
1,What does variable names beginning with _ mean?,<p>When writing my first asp.net MVC applicati...,<c#><asp.net><syntax><variables><naming-conven...
2,Is it valid to replace http:// with // in a <s...,<p>I have the following element:</p>\n\n<pre><...,<html><http><https><uri><protocol-relative>
3,Will new return NULL in any case?,<p>I know that according to C++ standard in ca...,<c++><visual-c++><memory-management><new-opera...
4,Convert float to string in positional format (...,<p>I want to print some floating point numbers...,<python><python-3.x><floating-point><number-fo...
...,...,...,...
17667,Cross-Origin-Opener-Policy policy would block ...,<p>I am using firebase and its google auth too...,<firebase><next.js><oauth-2.0><firebase-authen...
17668,VSC PowerShell. After npm updating packages .p...,<p>I design websites in VSC and PowerShell is ...,<firebase><powershell><vue.js><npm><visual-stu...
17669,How to force GCC to assume that a floating-poi...,<p>There are cases where you know that a certa...,<c++><gcc><assembly><floating-point><micro-opt...
17670,Jupyter command `jupyter-lab` not found,<p>I have tried to install jupyter lab on my K...,<python><python-3.x><jupyter-notebook><jupyter...


In [5]:
df['text'] = df['Title'].apply(lambda s: BeautifulSoup(s).text) + ' - ' + df['Body'].apply(lambda s: BeautifulSoup(s).text)

/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
df

,Title,Body,Tags,text
0,efficient way to implement paging,<p>Should I use LINQ's <code>Skip()</code> and...,<sql><sql-server><asp.net-mvc><linq-to-sql><pa...,efficient way to implement paging - Should I u...
1,What does variable names beginning with _ mean?,<p>When writing my first asp.net MVC applicati...,<c#><asp.net><syntax><variables><naming-conven...,What does variable names beginning with _ mean...
2,Is it valid to replace http:// with // in a <s...,<p>I have the following element:</p>\n\n<pre><...,<html><http><https><uri><protocol-relative>,Is it valid to replace http:// with // in a -...
3,Will new return NULL in any case?,<p>I know that according to C++ standard in ca...,<c++><visual-c++><memory-management><new-opera...,Will new return NULL in any case? - I know tha...
4,Convert float to string in positional format (...,<p>I want to print some floating point numbers...,<python><python-3.x><floating-point><number-fo...,Convert float to string in positional format (...
...,...,...,...,...
17667,Cross-Origin-Opener-Policy policy would block ...,<p>I am using firebase and its google auth too...,<firebase><next.js><oauth-2.0><firebase-authen...,Cross-Origin-Opener-Policy policy would block ...
17668,VSC PowerShell. After npm updating packages .p...,<p>I design websites in VSC and PowerShell is ...,<firebase><powershell><vue.js><npm><visual-stu...,VSC PowerShell. After npm updating packages .p...
17669,How to force GCC to assume that a floating-poi...,<p>There are cases where you know that a certa...,<c++><gcc><assembly><floating-point><micro-opt...,How to force GCC to assume that a floating-poi...
17670,Jupyter command `jupyter-lab` not found,<p>I have tried to install jupyter lab on my K...,<python><python-3.x><jupyter-notebook><jupyter...,Jupyter command `jupyter-lab` not found - I ha...


In [7]:
nltk.download('stopwords')
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer which is developed in recent years
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marzime/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [9]:
i=0
str1=' '
final_string=[]
s=''
for sent in df['text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [10]:
df['clean_text']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
df['clean_text']=df['clean_text'].str.decode("utf-8")

In [11]:
df

,Title,Body,Tags,text,clean_text
0,efficient way to implement paging,<p>Should I use LINQ's <code>Skip()</code> and...,<sql><sql-server><asp.net-mvc><linq-to-sql><pa...,efficient way to implement paging - Should I u...,effici way implement page use linq skip take m...
1,What does variable names beginning with _ mean?,<p>When writing my first asp.net MVC applicati...,<c#><asp.net><syntax><variables><naming-conven...,What does variable names beginning with _ mean...,variabl name begin mean write first asp net mv...
2,Is it valid to replace http:// with // in a <s...,<p>I have the following element:</p>\n\n<pre><...,<html><http><https><uri><protocol-relative>,Is it valid to replace http:// with // in a -...,valid replac follow case site https site may a...
3,Will new return NULL in any case?,<p>I know that according to C++ standard in ca...,<c++><visual-c++><memory-management><new-opera...,Will new return NULL in any case? - I know tha...,new return null case know accord standard case...
4,Convert float to string in positional format (...,<p>I want to print some floating point numbers...,<python><python-3.x><floating-point><number-fo...,Convert float to string in positional format (...,convert float string posit format without scie...
...,...,...,...,...,...
17667,Cross-Origin-Opener-Policy policy would block ...,<p>I am using firebase and its google auth too...,<firebase><next.js><oauth-2.0><firebase-authen...,Cross-Origin-Opener-Policy policy would block ...,polici would block window close call error use...
17668,VSC PowerShell. After npm updating packages .p...,<p>I design websites in VSC and PowerShell is ...,<firebase><powershell><vue.js><npm><visual-stu...,VSC PowerShell. After npm updating packages .p...,vsc powershel npm updat packag cannot load run...
17669,How to force GCC to assume that a floating-poi...,<p>There are cases where you know that a certa...,<c++><gcc><assembly><floating-point><micro-opt...,How to force GCC to assume that a floating-poi...,forc gcc assum express case know certain expre...
17670,Jupyter command `jupyter-lab` not found,<p>I have tried to install jupyter lab on my K...,<python><python-3.x><jupyter-notebook><jupyter...,Jupyter command `jupyter-lab` not found - I ha...,jupyt command found tri instal jupyt lab kubun...


# Modeling

In [24]:
def vectorizer_info(vectorizer, vectorized):
    terms = vectorizer.get_feature_names_out()
    
    return vectorized.shape, vectorizer, terms[1:10]

def model_info(model, vectorizer):
    labels = model.labels_
    silhouette_score = metrics.silhouette_score(vectorizer, labels, metric='euclidean')
    print(silhouette_score)

def cluster_info(data, label: str, text: str, vectorizer, model, cluster_nbr: int):
    
    data_goruped = data.groupby([label])[text].count()

    print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()

    for i in range(cluster_nbr):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :cluster_nbr]:
            print(' %s' % terms[ind], end='')
            print()

    plt.bar([x for x in range(cluster_nbr)], data.groupby([label])[text].count(), alpha = 0.4)
    plt.title('Cluster points')
    plt.xlabel("Cluster number")
    plt.ylabel("Number of points")
    plt.show()

    for i in range(cluster_nbr):
        print("A review of assigned to cluster ", i)
        print("-" * 70)
        print(data.iloc[data.groupby([label]).groups[i][0]][text])
        print('\n')
        print("_" * 70)
        
    return data_goruped

def silhouette(vectorized_data):
    for i in range(2,10):
    model = KMeans(n_clusters = i, init='k-means++',random_state=99)
    model.fit(vectorized_data)
    print(f"for {i} clusters")
    model_info(model, vectorized_data)

In [13]:
count_vect = CountVectorizer()

In [14]:
bow = count_vect.fit_transform(df['clean_text'].values)

In [15]:
vectorizer_info(count_vect, bow)

((17672, 44518),
 CountVectorizer(),
 array(['aaa', 'aaaa', 'aaaaa', 'aaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaa',
        'aaaaaaaaaaaaaaaaaaaaaaaaaaaab', 'aaaaaaaaavu', 'aaaabbbbbccddddd',
        'aaaabbbc'], dtype=object))

In [23]:
silhouette(bow)

/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[5.65898930e-05 1.07520797e-03 1.69769679e-04 ... 5.65898930e-05
  5.65898930e-05 5.65898930e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
0.9655476478970229


/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[5.66187295e-05 1.07575586e-03 1.69856188e-04 ... 5.66187295e-05
  5.66187295e-05 5.66187295e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
0.9075760202898117


/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.68117259e-05 1.07942279e-03 1.70435178e-04 ... 5.68117259e-05
  5.68117259e-05 5.68117259e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
0.7961990320053612


/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[5.92557478e-05 9.48091965e-04 1.77767243e-04 ... 5.92557478e-05
  0.00000000e+00 5.92557478e-05]
 [0.00000000e+00 3.95778364e-03 0.00000000e+00 ... 0.00000000e+00
  1.31926121e-03 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
0.6130440275087023


/home/marzime/.pyenv/versions/3.8.12/envs/simplon/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
model = KMeans(n_clusters = 10, init='k-means++',random_state=99)
model.fit(bow)

In [ ]:
model_info(model, bow)

In [ ]:
df['Bow Clus Label'] = model.labels_ # the last column you can see the label numebers
df

In [ ]:
cluster_info(df, 'Bow Clus Label', 'text', count_vect, model, 10)

# TF-IDF

In [ ]:
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(df['Text'].values)

In [ ]:
vectorizer_info(tfidf)

In [ ]:
silhouette(tfidf)

In [ ]:
model_tf = KMeans(n_clusters = 10, init='k-means++',random_state=99)
model_tf.fit(tfidf)

In [ ]:
model_info(model_tf, tfidf)

In [ ]:
df['Tfidf Clus Label'] = model_tf.labels_
df

In [ ]:
cluster_info(data, 'Tfidf Clus Label', 'Text', tfidf, model_tf, 10)